In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
torch.__version__

'1.10.0+cu111'

# logistic回归实战
在这一章里面，我们将处理一下结构化数据，并使用logistic回归对结构化数据进行简单的分类。
##  logistic回归(logistic regression)介绍
`logistic回归`是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有` wx + b，其中w和b是待求参数`，其区别在于他们的因变量不同，`多重线性回归`直接`将wx+b作为因变量`，即`y =wx+b`,而`logistic回归`则通过函数L将wx+b对应一个隐状态p，`p =L(wx+b`然后`根据p 与1-p的大小决定因变量的值`。如果`L是logistic函数，就是logistic回归`，如果L是多项式函数就是多项式回归。

即：logistic regression在计算了wx+b后，还要经过一层函数（吴恩达机器学习中为sigmoid函数，实际上也可以是softmax函数），将连续值转化为标签概率值

说的更通俗一点，就是logistic回归会在`线性回归后再加一层logistic函数`的调用。

logistic回归主要是进行二分类预测，我们在激活函数时候讲到过 Sigmod函数，`Sigmod函数是最常见的logistic函数`，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0。

下面我们就来使用公开的数据来进行介绍

## UCI German Credit  数据集

UCI German Credit是UCI的德国信用数据集，里面有原数据和数值化后的数据。

German Credit数据是根据个人的银行贷款信息和申请客户贷款逾期发生情况来预测贷款违约倾向的数据集，数据集包含24个维度的，1000条数据，

在这里我们直接使用处理好的数值化的数据，作为展示。

[地址](https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/)

## 代码实战
我们这里使用的 german.data-numeric是numpy处理好数值化数据，我们直接使用numpy的load方法读取即可

In [ ]:
data=np.loadtxt("/content/drive/MyDrive/pytorch-handbook/chapter3(practice )/german.data-numeric")

数据读取完成后我们要对数据做一下归一化的处理

In [ ]:
n,l=data.shape ## n为行数，l为列数
## 对每一列用z-score来归一化
for j in range(l-1):
    meanVal=np.mean(data[:,j])
    stdVal=np.std(data[:,j])
    data[:,j]=(data[:,j]-meanVal)/stdVal

打乱数据

In [ ]:
np.random.shuffle(data)

区分训练集和测试集，由于这里没有验证集，所以我们直接使用测试集的准确度作为评判好坏的标准。

区分规则：900条用于训练，100条作为测试

german.data-numeric的格式为，前24列为24个维度，最后一个为要打的标签（0，1）（y），所以我们将数据和标签一起区分出来

In [ ]:
## 定义training set 和 test set
train_data=data[:900,:l-1] ## 1-900行
test_data=data[900:,:l-1] ## 901-1000

train_lab=data[:900,l-1]-1 ## 对应train_data的最后一列
test_lab=data[900:,l-1]-1 ## 减一是因为数据集的label是1-2，但sigmoid接受的范围是0-1

In [ ]:
## 下面我们定义模型
class LogisticRegression(nn.Module):
  def __init__(self):
    ## super().__init__()
    super(LogisticRegression,self).__init__() ## 继承nn.Module中的__init__方法，在调用子类LogisticRegression的__init__的同时，还会调用父类nn.Module中的__init__的内容
    self.fc=nn.Linear(24,2) ## 这里的24和dataset的features一致,2对应二分类的两个概率

  def forward(self,x): ## x 为 input
    out=self.fc(x) ## 经过一层线性层，得到多元回归的结果
    out=torch.sigmoid(out) ## 经过sigmoid激活，将多元回归的结果转化为标签
    return out

def test(pred,lab):
    t=pred.max(-1)[1]==lab ## 每个实例预测最大的预测概率
    return torch.mean(t.float())

## 其实这里比较奇怪，按照吴恩达在机器学习中的降解，logistic regression的h(x)是预测y=1时的概率，这里定义网络实际上定义成nn.Linear(24,1)就可以了

In [ ]:
## super() 介绍
class A():
    def fortest(self):
        print('Call class A')
        print('Leave class A')

class B1(A):
    def fortest(self):
        print('Call class B1')
        print('Leave class B1')


class B2(A): ## 有继承
    def fortest(self):
        print('Call class B2')
        super(B2,self).fortest()
        print('Leave class B2')
sample1=B1()
sample2=B2()

print(sample1.fortest()) ## 没有继承，所以调用sample1对应的子类B1，只调用自己的fortest方法（父类的fortest方法被覆盖）
print('\n')
print(sample2.fortest()) ## 继承了，调用子类B2的fortest方法的同时还调用了父类的fortest方法，不会覆盖父类

Call class B1
Leave class B1
None


Call class B2
Call class A
Leave class A
Leave class B2
None


In [ ]:
## 初始化网络
net=LogisticRegression() 
criterion=nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm=torch.optim.Adam(net.parameters()) # Adam优化
epochs=1000 # 训练1000次

In [ ]:
## training
for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    # 输入值都需要转化成torch的Tensor
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))
    ## 训练完成

Epoch:100,Loss:0.6475,Accuracy：0.71
Epoch:200,Loss:0.6187,Accuracy：0.76
Epoch:300,Loss:0.6006,Accuracy：0.77
Epoch:400,Loss:0.5882,Accuracy：0.77
Epoch:500,Loss:0.5789,Accuracy：0.77
Epoch:600,Loss:0.5717,Accuracy：0.76
Epoch:700,Loss:0.5658,Accuracy：0.77
Epoch:800,Loss:0.5609,Accuracy：0.78
Epoch:900,Loss:0.5567,Accuracy：0.77
Epoch:1000,Loss:0.5531,Accuracy：0.77


In [ ]:
## some details
a=test_out[0:5,:]
print(a,'\n')

## .max()
print(a.max(0),'\n') ## .max(0)，对每列求最值
print(a.max(1),'\n') ## .max(1)，对每行求最值
print(a.max(1)[0],'\n')

tensor([[0.8997, 0.2912],
        [0.3434, 0.7658],
        [0.9145, 0.0871],
        [0.5292, 0.4603],
        [0.9836, 0.0072]], grad_fn=<SliceBackward0>) 

torch.return_types.max(
values=tensor([0.9836, 0.7658], grad_fn=<MaxBackward0>),
indices=tensor([4, 1])) 

torch.return_types.max(
values=tensor([0.8997, 0.7658, 0.9145, 0.5292, 0.9836], grad_fn=<MaxBackward0>),
indices=tensor([0, 1, 0, 0, 0])) 

tensor([0.8997, 0.7658, 0.9145, 0.5292, 0.9836], grad_fn=<MaxBackward0>) 

